## Лабораторная работа №1

1. Файл `minutes_n_ingredients.csv` содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива `numpy` типа `int32`, используя `np.loadtxt`. Выведите на экран первые 5 строк массива.

In [18]:
import numpy as np
data = np.loadtxt('minutes_n_ingredients.csv', delimiter=',', skiprows=1, dtype=np.int32)
print(data[:5])

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 [ 67660      5      6]
 [157911     60     14]]


2. Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [19]:
print(data[:, 1:].mean(axis=0)) # среднее значение
print(data[:, 1:].min(axis=0)) # минимум 
print(data[:, 1:].max(axis=0)) # максимум 
print(np.median(data[:, 1:], axis=0)) # медиана


[2.16010017e+04 9.05528000e+00]
[0 1]
[2147483647         39]
[40.  9.]


3. Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля $q_{0.75}$. 

In [20]:
border = np.quantile(data[:, 1], q=0.75)
data[:, 1] = data[:, 1].clip(max=border)
print(border)

65.0


4. Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [21]:
print(data[data[:, 1]==0].shape[0])
data[data[:, 1]==0, 1]=1

479


5. Посчитайте, сколько уникальных рецептов находится в датасете.

In [22]:
print(len(np.unique(data[:, 0], axis=0)))

100000

6. Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [25]:
print(len(np.unique(data[:, 2])))
np.unique(data[:, 2])

37


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 37, 39], dtype=int32)

7. Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [27]:
data_v2 = data[data[:, 2] <= 5].copy()
data_v2

array([[446597,     15,      5],
       [204134,      5,      3],
       [ 25623,      6,      4],
       ...,
       [ 52088,     60,      5],
       [128811,     15,      4],
       [370915,      5,      4]], dtype=int32)

8. Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [28]:
average_ingr_on_min = data[:, 2] / data[:, 1]
average_ingr_on_min.mean(), average_ingr_on_min.max()

(0.3983000355850672, 24.0)

9. Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [29]:
print(data[data[:, 1].argsort()][-100:][:, 2].mean())

9.96


10. Выберите случайным образом и выведите информацию о 10 различных рецептах

In [32]:
random = np.random.randint(0, 999, size=10)
data[random]

array([[ 87775,     65,     13],
       [256227,     65,     10],
       [157911,     60,     14],
       [143195,     65,     10],
       [204134,      5,      3],
       [ 69317,     10,      8],
       [276123,     60,      7],
       [276123,     60,      7],
       [308301,     65,     11],
       [ 74205,     20,      7]], dtype=int32)

11. Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [33]:
less_than_average = data[:, 2].mean()
len(data[data[:, 2] < less_than_average]) / len(data) * 100

58.802

12. Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.

In [34]:
simple = (data[:, 1] <= 20) & (data[:, 2] <= 5)
simple  = simple .astype(np.int32)
simple  = simple [:, np.newaxis]

simple_data  = np.hstack((data, simple ))
simple_data

array([[127244,     60,     16,      0],
       [ 23891,     25,      7,      0],
       [ 94746,     10,      6,      0],
       ...,
       [498432,     65,     15,      0],
       [370915,      5,      4,      1],
       [ 81993,     65,     14,      0]], dtype=int32)

13. Выведите процент "простых" рецептов в датасете

In [ ]:
print(len(simple_data[simple_data[:, 3] == 1]) / len(simple_data) * 100)

14. Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.

In [35]:
short = data[data[:, 1] < 10]
standard = data[(data[:, 1] >= 10) & (data[:, 1] < 20)]
Long = data[data[:, 1] >= 20]
stop = min([len(arr) for arr in [short, standard, Long]])
np.array([short[:stop], standard[:stop], Long[:stop]])

array([[[ 67660,      5,      6],
        [366174,      7,      9],
        [204134,      5,      3],
        ...,
        [420725,      5,      3],
        [  4747,      1,      9],
        [370915,      5,      4]],

       [[ 94746,     10,      6],
        [ 33941,     18,      9],
        [446597,     15,      5],
        ...,
        [  9831,     15,      7],
        [335859,     12,     14],
        [256812,     10,      3]],

       [[127244,     60,     16],
        [ 23891,     25,      7],
        [157911,     60,     14],
        ...,
        [168901,     25,      7],
        [392339,     35,     13],
        [206732,     45,     10]]], dtype=int32)